# Predict Future Sales
### Nicklas Ankarstad

In [ ]:
### ADD INTRO HERE

In [2]:
import pandas as pd
import numpy as np
from googletrans import Translator
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
%matplotlib inline
from matplotlib import pyplot
from itertools import product
import time
from sklearn.model_selection import KFold
from sklearn import base
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, KFold, train_test_split
import pickle
import calendar
from datetime import datetime

## The Data

This contest has several data files that we will have to join together. For those with database modeling understanding, the sales_train file or training set can be thought of as the fact table in a star schema with items, item_ categories and shops being dimensional tables we can join to with a primary key. The test file is another fact with similar relationships. The key difference between the sales_train file and the test file is that the sales file is __daily__ and the test file is __monthly__. This means we will also have to aggregrate daily data to monthly. 

__sales_train.csv__ - the training set. Daily historical data from January 2013 to October 2015.

__test.csv__ - the test set. You need to forecast the sales for these shops and products for November 2015.

__sample_submission.csv__ - a sample submission file in the correct format.

__items.csv__ - supplemental information about the items/products.

__item_categories.csv__  - supplemental information about the items categories.

__shops.csv__- supplemental information about the shops.

In [ ]:
## allows us to pick up the european formatting of the dates in the trainset
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y') 
# importing the trainset with dates correctly formatted
sales = pd.read_csv('sales_train.csv.gz', parse_dates = ['date'], date_parser = dateparse)
#import the rest of the files
test = pd.read_csv('test.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

Now that we have read in all the files. Let's start analyzing them one by one. Starting with the Item Categories.

## Item Categories file

The file is unfortanately in Russian, but based on the names of the columns this file can help us understand what the categories of the products refer to. In this section we will use googletrans to translate the category names to English where-ever possible and then eventually derive a new feature that we will call __Category_type__

In [ ]:
### Start by checking out the top five records
item_categories.head()

Not speaking Russian can be a disadvantaged, but to the (non-russian) naked eye it appears as though we have a patterns where there is a word followed by PS2, PS3, PS4 and PSP. That looks a lot like it related to various playstation platforms. Maybe the first can help us caetgories these items together. Let's translate the words and see.

In [ ]:
# Starting with translating the column item_category name from Russian to English. We will then append that to the original dataframe.
translator = Translator()
list_a = []
for word in item_categories['item_category_name']:
    try:
        a = translator.translate(word).text
        list_a.append(a)
    except:
        list_a.append(word)
item_categories['English_Name'] = list(list_a)
    #print(list_a)

The translator is not perfect as it missed some terms. We can manually replace them so our categories are easier to work with.

In [ ]:
## Программы means Programs
item_categories['English_Name']= item_categories['English_Name'].str.replace("Программы", "Programs")

## Книги means Books
item_categories['English_Name']= item_categories['English_Name'].str.replace("Книги", "Books")

item_categories.head()


Aha! Looks like the first part of the of each item category has the category type in it.  In our example, the category type was accessories. Let's extract that out and store it in a new feature called Category_type.


In [ ]:
## Create a feature called Variable type by splitting the English_Name strings where they either have a paranthesis or a dash.
list_a = []
for row in item_categories['English_Name']:
        a = row.replace('(','-').split(' -')[0] ## replacing the opening parantheses with dash so we can use str.split function to split on it.
        list_a.append(a)
item_categories['Category_type'] = list(list_a)
## Lets check out the categories we have
pd.DataFrame((item_categories['Category_type'].unique()))

Looks like several of the categories have similar names and meaning. For example, __game__ console and __gaming__ console are virtually the same type of category. Let's clean those up a bit and get more uniformity of this new feature.



In [ ]:
## Let's clean up some of this output in the categories:

## Game Consoles are really the same thing as Gaming Consoles
item_categories['Category_type']= item_categories['Category_type'].str.replace("Gaming Consoles", "Game Consoles")

## Payment cards with a lowercase c is the same as Payment Cards with upper case C
item_categories['Category_type']= item_categories['Category_type'].str.replace("Payment cards", "Payment Cards")

## Cinema and movie tends to be synomomous. Let's change "The Movie" category type to Cinema
item_categories['Category_type']= item_categories['Category_type'].str.replace("The Movie", "Cinema")

## Pure and Clean Media Seem Similar. Let's combine into Pure/Clean Media
item_categories['Category_type']= item_categories['Category_type'].str.replace("Clean media", "Pure/Clean Media")
item_categories['Category_type']= item_categories['Category_type'].str.replace("Pure Media", "Pure/Clean Media")

## Lets check out the categories we have
pd.DataFrame((item_categories['Category_type'].unique()))


That looks better. Since this dataset is on the larger side (for a laptop to handle anyways), let's drop the columns we are not going to be using. This will allow us to use less memory on the laptop.


In [ ]:
## Lets drop item categories_name, the English name and leave only category type

item_categories = item_categories.drop(['item_category_name','English_Name'],axis =1)

That's better. Looks like this dataframe is ready to be used. Let's move on to the shop/store dataframe before we merge all the data together.

## Shops File

This file contains the names of the shops. It can be used as a key to get the names for the shop Id's we have in the sales file. Because this file is also in Russian, we will again translate the words into English. Once we have the names in English, we will extract the cities these shops are located within and use that as a feature.

In [ ]:
shops.head()

Despite seeing learning about how to spell accesories in Russian, I am afraid my Russian is still not good enough to read the shop names. Let's translate them to English and take a look at what the words mean.

In [ ]:
## Let's translate this into English
translator = Translator()
list_a = []
for word in shops['shop_name']:
    a = translator.translate(word).text
    list_a.append(a)
shops['English_Shop_Name'] = list_a
    #print(list_a)

In [ ]:
shops

Looks like the city is the first word, followed by something like shopping center, TC or ,SEC. Let's try and extract the city from this. Some googeling of the words made it seem like all the spots I checked, whether they are TC or SEC were in shopping malls. As a result we did not create a featured out that part of the shop name


In [ ]:
## Create city variable with only the city names (first word)
## Create a feature called City type by splitting the English_Shops_Name strings where by the spaces. Because there are some cities like St. Petersburg that have a space in their name, we remove spaces following a period and spaces following an exclamation point

list_a = []
for row in shops['English_Shop_Name']:
        a = row.replace('. ','').replace('! ','').split(' ')[0] ## remove spaces follwing period or exclaimation point and split based on spaces. First word is city
        list_a.append(a)
shops['City'] = list(list_a)
## Lets check out the categories we have
pd.DataFrame((shops['City'].unique()))

In [ ]:
shops.head()

## Question: Are these the same shop?

Жуковский ул. Чкалова 39м? and  Жуковский ул. Чкалова 39м²	 
Shop_Id 10 and 11

!Якутск ТЦ "Центральный" фран and Якутск ТЦ "Центральный"	
shop ID 01 and 58

!Якутск Орджоникидзе, 56 фран and Yakutsk Ordzhonikidze, 56

shop ID 0 and 59

In [ ]:
## Filter a down to the two shop_id's we are interested in, group by both the id's and time while adding up all the item_cnt_day
## Then plot it

## Filter down to the two shops
a = sales[(sales['shop_id'] == 10) | (sales['shop_id'] == 11)]
## Group and sum the item_cnt_by shop and date block.
a = a.groupby(['shop_id','date_block_num']).sum().reset_index()

## Seaborne doesn't like when you covert integers and floats to strings and pass them as colors so we add some extra text to trick it to thinking its really a string.

rownumb= 0
for row in a['shop_id']:
    a.loc[rownumb,'shop_id'] = "shop_id: %r" % (row)
    rownumb = rownumb+1
## PLot it.
a4_dims = figsize =  (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
#sns.lineplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None, ax = ax)
sns.barplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None, ax = ax)

In [ ]:
a4_dims = figsize =  (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)

sns.barplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None, ax = ax)

This is interesting. We can see that for shop 10 had sales for every time block except 25 and the inverse is true for shop 11. This tells us that they are the same. We will replace any reference to 11 with 10 in the sales dataframe.

In [ ]:
## Replace 11 with 10
sales['shop_id'] = sales['shop_id'].replace(11,10)

In [ ]:
## Filter a down to the two shop_id's we are interested in, group by both the id's and time while adding up all the item_cnt_day
## Then plot it

## Filter down to the two shops
a = sales[(sales['shop_id'] == 1) | (sales['shop_id'] == 58)]
## Group and sum the item_cnt_by shop and date block.
a = a.groupby(['shop_id','date_block_num']).sum().reset_index()

## Seaborne doesn't like when you covert integers and floats to strings and pass them as colors so we add some extra text to trick it to thinking its really a string.

rownumb= 0
for row in a['shop_id']:
    a.loc[rownumb,'shop_id'] = "shop_id: %r" % (row)
    rownumb = rownumb+1
## PLot it.
#sns.lineplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None)


a4_dims = figsize =  (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)

sns.barplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None, ax = ax)


Looks like we found another. Started as number 1 and continued as number 58. Let's replace 1 with 58


In [ ]:
## Replace 1 with 58
sales['shop_id'] = sales['shop_id'].replace(1,58)

In [ ]:
## Filter a down to the two shop_id's we are interested in, group by both the id's and time while adding up all the item_cnt_day
## Then plot it

## Filter down to the two shops
a = sales[(sales['shop_id'] == 0) | (sales['shop_id'] == 59)]
## Group and sum the item_cnt_by shop and date block.
a = a.groupby(['shop_id','date_block_num']).sum().reset_index()

## Seaborne doesn't like when you covert integers and floats to strings and pass them as colors so we add some extra text to trick it to thinking its really a string.

rownumb= 0
for row in a['shop_id']:
    a.loc[rownumb,'shop_id'] = "shop_id: %r" % (row)
    rownumb = rownumb+1
## PLot it.
#sns.lineplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None)

a4_dims = figsize =  (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)

sns.barplot(x = 'date_block_num', y ='item_cnt_day', hue = 'shop_id', data = a, ci = None, ax = ax)

This looks we have two different shops since shop 0 was having around 3 times as many sales per date block compared to shop 59. Both have sales in the same month as well. Let's leave this one.

In [ ]:
### Lets drop the shop_name, English Shop Name since we extracted the city information from it already
shops = shops.drop(['shop_name','English_Shop_Name'],axis = 1)
shops.head()

## Aggregate data
Since the competition task is to make a monthly prediction, we need to aggregate the data to monthly level before doing any encodings. The following code-cell serves just that purpose. It also renames the item_cnt_day varibale into Target (once we have made it a monthly aggregate).

In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

Another key component of this competition is the need to set a maximum of 20 for each monthly target. This means that if we see a value larger than 20, we will automatically call it 20. This has a significant positive impact on our RMSE score.

In [ ]:
all_data['target']=all_data['target'].clip(0,20)

In [ ]:
all_data.head()

## Merge Datasets Together

Let's create one dataframe with our train and test sets. We will join this with the item_categories, items and shops. We will use this dataframe to create alot of our features and reduce the need to apply them to multiple dataframes wherever possible. For example, when we create lagged variables, we need to create them for both the train, validation and test sets. By combining all the data into one dataframe we only have to do this once.

In [ ]:
## Let's check out the shape of our datasets
all_data.shape , test.shape

We will union train and test sets together. As we saw from the code right above, we are missing two columns on the test set. These are date_block_num and target. For now we will assign the target to be zero. We will also assign the number 34 to the date_block_num. The date_block_num corresponds to the month in the dataset, so since we need to predict next months item_counts, we will simply look for the max of the training set and add one. (the max is 33)

In [ ]:
## Assign 34 to date_block_num and 0.0 to target
test['date_block_num'] = 34
test['target'] = 0.0

TEST_ID = test['ID'] ## in case we need this later

## Then we need to union them and save that back as our all_data dataframe
all_data = pd.concat([all_data,test], axis =0, sort=True)
all_data = all_data.drop(columns = ['ID'])


Next we will merge the all_data dataframe with the items, item_categories and shops dataframes. Since we want to avoid kartesian products so we will add a number of row counts checker to make sure we do not add any new rows or drop any.

In [ ]:
## Calculate number of rows prior to merge
prior_rows = all_data.shape[0]

## Merge the sales train data with the items, item categoris and shops datasets to get the names of items, their categories and the shop names
all_data = pd.merge(all_data, items, on = "item_id")
all_data = pd.merge(all_data, item_categories, on = "item_category_id")
all_data = pd.merge(all_data, shops, on = "shop_id")

## Calcualte number and print of rows dropped (should be zero)
print("Dropped {} rows".format(prior_rows - all_data.shape[0]))


In [ ]:

# import calendar
# from datetime import datetime

# len([1 for i in calendar.monthcalendar(datetime.now().year,
#                                   datetime.now().month) if i[6] != 0])

## Dates

Dates can tell us a lot of things about sales. For example, February sales may be lower than January sales simply because February has fewer days in it (ie less time to sell). The types of days also matter. More weekend days may mean that more people frequent the stores. Seasons matter as well, where June sales may be different than Decembers. We will create features related to all of these items.

To get started, we first need to extract the Month-End Dates for each dateblock and store it in a dataframe.

In [ ]:
## Pull out the last date of each dateblock and append it to the 

list_a = []
for dateblock in sales['date_block_num'].unique():
    a = sales[sales['date_block_num'] == dateblock]
    a = max(a['date'])
    list_a.append(a)
    
list_a.append(datetime.strptime('2015-11-30','%Y-%m-%d')) ## Manually adding the month for the test set
## Transform it to dataframe so we can merge with all_data
list_a = pd.DataFrame(list_a)
## Give the data a descriptive column header
list_a.columns = ['Month_End_Date'] 


Now that the Month End Dates have been extracted, we can count the number of mondays, tuesdays etc there were in each month.

In [ ]:
## Let's calculate the number of specific days are in each month.

## Create the empty lists
mon_list = []
tue_list = []
wed_list = []
thu_list = []
fri_list = []
sat_list = []
sun_list = []

## Calculate the number of a specific day in a given month (for example, number of mondays in March of 2015)
for date in list_a['Month_End_Date']:
    mon_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[0] != 0])))
    tue_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[1] != 0])))
    wed_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[2] != 0])))
    thu_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[3] != 0])))
    fri_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[4] != 0])))
    sat_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[5] != 0])))
    sun_list.append((len([1 for i in calendar.monthcalendar(date.year,date.month) if i[6] != 0])))

## Add these to our list we created with the dates
list_a['Number_of_Mondays'] = mon_list
list_a['Number_of_Tuesdays'] = tue_list
list_a['Number_of_Wednesdays'] = wed_list
list_a['Number_of_Thursdays'] = thu_list
list_a['Number_of_Fridays'] = fri_list
list_a['Number_of_Saturdays'] = sat_list
list_a['Number_of_Sundays'] = sun_list



We can also extract features related to the year, the month and the number of days in the month.

In [ ]:
## Create the empty lists again

year_list = []
month_list = []
day_list = []

## Next lets calculate strip out the number of days in a month, the number of the month and the number of the year
for date in list_a['Month_End_Date']:
    year_list.append(date.year)
    month_list.append(date.month)
    day_list.append(date.day)

## Add to our dataframe
list_a['Year'] = year_list
list_a['Month'] = month_list
list_a['Days_in_Month'] = day_list

Now we have a nice date dataframe called list_a.

In [ ]:
list_a.head()

The list_a dataframe can be merged back with the all_data dataframe and we've added a couple of date features.

In [ ]:
## Merge the new dataframe with the all_data, using the index and the date_block_num as keys
all_data = pd.merge(all_data, list_a, left_on = 'date_block_num', right_index = True)

all_data.head()

## Price

Our original approach was to add the count of transactions and we did not do anything with Price of the item. Let's average the monthly price and merge that feature back with our all_data dataframe.

In [ ]:
## adding the average monthly price within a monthly block for each item at each store to the dataset
a = sales.groupby(['date_block_num','shop_id','item_id'])['item_price'].mean()
a = pd.DataFrame(a)
all_data = pd.merge(all_data,a,how = "left", left_on = ['date_block_num','shop_id','item_id'], right_on = ['date_block_num','shop_id','item_id'])

### Months Since Item First Sold & Months Since Item was Last Sold

These features show the number of date blocks (months) since the first time the item was sold and the last time the item was sold. These will help us understand how new the item is and could potentially tell us that the item is no longer being sold.

In [ ]:
### Let's calculate how "fresh" the items are. We will calculate the min for each item. This will give us the first month it was sold in
### Then we will calculate the difference between that number and the current date block to see how "old" the item is.
a = all_data.groupby('item_id')['date_block_num'].min()
a = pd.DataFrame(a)
a = a.reset_index()
a.columns = ['item_id','min_item_sale_date_block_num']
all_data = pd.merge(all_data,a, left_on = 'item_id', right_on = 'item_id')
all_data['Months_Since_Item_First_Sold'] = all_data['date_block_num']- all_data['min_item_sale_date_block_num']


# ### Let's calculate how "stale" the items are. We will calculate the max for each item. This will give us the first month it was sold in
# ### Then we will calculate the difference between that number and the current date block to see how "old" the item is.
# a = all_data.groupby('item_id')['date_block_num'].max()
# a = pd.DataFrame(a)
# a = a.reset_index()
# a.columns = ['item_id','max_item_sale_date_block_num']
# all_data = pd.merge(all_data,a, left_on = 'item_id', right_on = 'item_id')
# all_data['Months_Since_Item_Last_Sold'] = all_data['date_block_num']- all_data['max_item_sale_date_block_num']

Some of the data in the test set are for products we have never seen before. Lets create a features that calculated the average monthly sales for a specific item only in the first month it was sold. We will make the rest of them zero. 

We will also apply the same logic to item categories and shop ids combined

In [ ]:
## calculate the average sales in the first month by category
a = all_data[all_data['Months_Since_Item_First_Sold'] == 0].groupby(['item_category_id','Months_Since_Item_First_Sold'])['target'].mean()
a = pd.DataFrame(a)
a = a.reset_index()
a.columns = ['item_category_id','Months_Since_Item_First_Sold','avg_first_months_sales_by_item_category_id']
all_data = pd.merge(all_data,a, left_on = ['item_category_id','Months_Since_Item_First_Sold'], right_on = ['item_category_id','Months_Since_Item_First_Sold'], how = 'left')
all_data['avg_first_months_sales_by_item_category_id'] = all_data['avg_first_months_sales_by_item_category_id'].fillna(0)


In [ ]:
## calculate the average sales in the first month by category and shop ID
a = all_data[all_data['Months_Since_Item_First_Sold'] == 0].groupby(['item_category_id', 'Months_Since_Item_First_Sold','shop_id'])['target'].mean()
a = pd.DataFrame(a)
a = a.reset_index()
a.columns = ['item_category_id','Months_Since_Item_First_Sold','shop_id','avg_first_months_sales_by_item_category_and_shop']
all_data = pd.merge(all_data,a, left_on = ['item_category_id','Months_Since_Item_First_Sold','shop_id'], right_on = ['item_category_id','Months_Since_Item_First_Sold', 'shop_id'], how = 'left')
all_data['avg_first_months_sales_by_item_category_and_shop'] = all_data['avg_first_months_sales_by_item_category_and_shop'].fillna(0)   

In [ ]:
# ## calculate the average sales in the first month by category
# a = all_data[all_data['Months_Since_Item_First_Sold'] == 0].groupby(['item_category_id','Months_Since_Item_First_Sold'])['target'].mean()
# a = pd.DataFrame(a)
# a = a.reset_index()
# a.columns = ['item_category_id','Months_Since_Item_First_Sold','avg_first_months_sales_by_item_category_id']
# all_data = pd.merge(all_data,a, left_on = ['item_category_id','Months_Since_Item_First_Sold'], right_on = ['item_category_id','Months_Since_Item_First_Sold'], how = 'left')
# all_data['avg_first_months_sales_by_item_category_and_shop'] = all_data['avg_first_months_sales_by_item_category_and_shop'].fillna(0)

In [ ]:
all_data[all_data['Months_Since_Item_First_Sold'] == 0]

Let's check out our dataframe

In [ ]:
all_data.head()

## Lagged Variables

If I was only allowed one datapoint to predict what next month's sales would be, I would probably use this months sales. This month's sale is a lagged 1 variable. Lags of target variables are very common in Time Series (Luckily we are not limited to only one data point). We will create a couple of lagged variables (ie last months sales, last (last) month's sales etc. We will repead this for several months. 

To set up the lagging features we will use a helper function (kindly provided by https://www.kaggle.com/dlarionov/feature-engineering-xgboost)



In [ ]:
## Create a helper function that allow us to pass what dataset we want to perform the lag on (df), 
## the lags we want to do and the column we want to lag
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

Using the function we just created, we will lag the target 1,2,3,4,5,6 and 12 months

In [ ]:
ts = time.time()
all_data = lag_feature(all_data, [1,2,3,4,5,6,12], 'target')
time.time() - ts

MORE! Lagging is fun. Let's create features using the mean for each month and either category, city, shop or item. We can use the lags of these features in our model. 

In [ ]:
## Average number of sales by month and by item
all_data['avg_monthly_by_item'] = all_data.groupby(['item_id', 'date_block_num'])['target'].transform('mean')

## Average number of sales by month and by shop
all_data['avg_monthly_by_shop'] = all_data.groupby(['shop_id', 'date_block_num'])['target'].transform('mean')

## Average number of sales by month and by category
all_data['avg_monthly_by_category'] = all_data.groupby(['Category_type', 'date_block_num'])['target'].transform('mean')

## Average number of sales by month and by city
all_data['avg_monthly_by_city'] = all_data.groupby(['City', 'date_block_num'])['target'].transform('mean')



### Let's lag the variables we just created using the lag_feature helper function. We will also lag the sale price since its not in the test set.

ts = time.time()
all_data = lag_feature(all_data, [1,2,3,6,12], 'avg_monthly_by_item')
all_data = lag_feature(all_data, [1,2,3,6,12], 'avg_monthly_by_shop')
all_data = lag_feature(all_data, [1,2,3,6,12], 'avg_monthly_by_category')
all_data = lag_feature(all_data, [1,2,3,6,12], 'avg_monthly_by_city')
all_data = lag_feature(all_data, [1,2,3,4,5,6], 'item_price')
time.time() - ts

## Dropping these as they would be NA's for the test set.
all_data = all_data.drop(['avg_monthly_by_item','avg_monthly_by_shop','avg_monthly_by_category','avg_monthly_by_city','item_price'], axis =1)

These lagged variables had a lot of Nulls as we didn't always have a sale in prior months. Let's make check how many we have before we assign them all to zero.

In [ ]:
null_list = all_data.isnull().sum(axis=0) ## sum of all the nulls. Store them in a pandas series object
null_df = pd.DataFrame(null_list) ## Convert to data frame so we can easily filter
null_df[null_df[0] > 0] ## Show only the columns with nulls

Looks like its only the lagged variables. A Null here really means we had zero sales. So lets make them Zero!

In [ ]:
for column in all_data:
    all_data[column] = all_data[column].fillna(0)

Another feature of interest is rolling or moving averages. Let's create two rolling averages, 3 and 6 months.

In [ ]:
## Calculate rolling average
all_data['target_3_month_avg'] = (all_data['target_lag_1'] + all_data['target_lag_2'] +all_data['target_lag_3']) /3
all_data['target_6_month_avg'] = (all_data['target_lag_1'] + all_data['target_lag_2'] +all_data['target_lag_3'] + all_data['target_lag_4'] + all_data['target_lag_5'] +all_data['target_lag_6']) /6

In [ ]:
all_data.head()

Wow that is a lot of features we have created.  Since I am only working with a laptop, we should probably make sure we are storing this in as small of dataframe as possible. Let's check the memory usage and the datatypes of our dataframe.

In [ ]:
all_data.info(memory_usage = "deep")

Since a lot of these are listed as either int64 or float64, we can probably reduce them down to smaller space datatypes like int16 or float8. Downcasting means we reduce the datatypes of each feature to its lowest possible type.

In [ ]:
for column in all_data:
    if all_data[column].dtype == 'float64':
        all_data[column]=pd.to_numeric(all_data[column], downcast='float')
    if all_data[column].dtype == 'int64':
        all_data[column]=pd.to_numeric(all_data[column], downcast='integer')
## Dropping Item name to free up memory
all_data = all_data.drop('item_name',axis =1)
## Let's check the size
all_data.info(memory_usage = "deep")

## Train Test Splitting

Now that our dataset has been downcast, we can start splitting data into training (first 32 months), validation (month 33) and back to our test set (month 34). 

In [ ]:
X_train = all_data[all_data.date_block_num < 33]
Y_train = all_data[all_data.date_block_num < 33]['target']
X_valid = all_data[all_data.date_block_num == 33]
Y_valid = all_data[all_data.date_block_num == 33]['target']
X_test = all_data[all_data.date_block_num == 34]

## Target Encoding

### Why do we target encode? 

Gradient boosted tree-based models such as XGBoost and LightGBM have a hard time handling high cardinality categorical variables. Target Encoding helps with that and can help improve the model performance. 

### Why do we need to reguralize?

Simply calculating the averages of the target variables can cause overfitting and often reduces the models ability to be generalized. 

#### Regularization Techniques:

 - Cross Validation Loop inside training data
 - Smoothing
 - Adding Random Noise
 - Sorting and Calculating expanding mean

We will only do cross-validation Loop inside our training data. To get started, we will define two helper function that I picked up from https://medium.com/@pouryaayria/k-fold-target-encoding-dfe9a594874b



In [ ]:
## Helpder Function to KFold Mean encoding
class KFoldTargetEncoderTrain(base.BaseEstimator,
                               base.TransformerMixin):
    def __init__(self,colnames,targetName,
                  n_fold=5, verbosity=True,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold,
                   shuffle = False, random_state=2019)
        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan
        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())
            X[col_mean_name].fillna(mean_of_target, inplace = True)
        if self.verbosity:
            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,self.targetName,                    
                   np.corrcoef(X[self.targetName].values,
                               encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X

    
## Helper function to get the Kfold Mean encoded on the test set

class KFoldTargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        mean =  self.train[[self.colNames,
                self.encodedName]].groupby(
                                self.colNames).mean().reset_index() 
        
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})
        return X

Now that we have both of our helper functions defined, lets use them to start mean-encoding our variables:
- item_id
- shop_id
- City
- Category_type
- item_category_id

In [ ]:
## Transforms the train set with a mean-encoded feature

## item_id mean encoding
targetc = KFoldTargetEncoderTrain('item_id','target',n_fold=5)
X_train = targetc.fit_transform(X_train)

## shop_id mean encoding
targetc = KFoldTargetEncoderTrain('shop_id','target',n_fold=5)
X_train = targetc.fit_transform(X_train)

## City mean encoding
targetc = KFoldTargetEncoderTrain('City','target',n_fold=5)
X_train = targetc.fit_transform(X_train)

## Category_type mean encoding
targetc = KFoldTargetEncoderTrain('Category_type','target',n_fold=5)
X_train = targetc.fit_transform(X_train)


## Item_category_id mean encoding
targetc = KFoldTargetEncoderTrain('item_category_id','target',n_fold=5)
X_train = targetc.fit_transform(X_train)

In [ ]:
## Transform validation & test set

## Apply item id mean encoding to test set
test_targetc = KFoldTargetEncoderTest(X_train,'item_id','item_id_Kfold_Target_Enc')
X_valid = test_targetc.fit_transform(X_valid)
X_test = test_targetc.fit_transform(X_test)

## Apply shop id mean encoding to test set
test_targetc = KFoldTargetEncoderTest(X_train,'shop_id','shop_id_Kfold_Target_Enc')
X_valid = test_targetc.fit_transform(X_valid)
X_test = test_targetc.fit_transform(X_test)


## Apply city mean encoding to test set
test_targetc = KFoldTargetEncoderTest(X_train,'City','City_Kfold_Target_Enc')
X_valid = test_targetc.fit_transform(X_valid)
X_test = test_targetc.fit_transform(X_test)

## Apply Category_type mean encoding to test set
test_targetc = KFoldTargetEncoderTest(X_train,'Category_type','Category_type_Kfold_Target_Enc')
X_valid = test_targetc.fit_transform(X_valid)
X_test = test_targetc.fit_transform(X_test)

## Apply item_category_id mean encoding to test set
test_targetc = KFoldTargetEncoderTest(X_train,'item_category_id','item_category_id_Kfold_Target_Enc')
X_valid = test_targetc.fit_transform(X_valid)
X_test = test_targetc.fit_transform(X_test)



### Final Dataset

We are getting close. Our fetures are done. Let' do a couple of checks to make sure we only have the features we will use.

In [ ]:
## drop first 12 months since we have lagged variables
X_train = X_train[X_train.date_block_num > 12]

## Assign target variables to seperate variables
y= X_train['target']
Y_valid = X_valid['target']


## Drop Categorical Variables that we mean encoded, the target and the item codes.
columns_to_drop = ['target', 'Category_type','City','Month_End_Date', 'item_category_id']
X_train= X_train.drop(columns_to_drop, axis = 1)
X_valid = X_valid.drop(columns_to_drop, axis = 1)
X_test = X_test.drop(columns_to_drop, axis = 1)


## Double check that our shapes are the same
X_train.shape,X_valid.shape, X_test.shape

### Save the files to a local drive using Pickle

As I mentioned before, memory is an issue. Here is an option to save the dataframes we have used thus far using pickle.

In [ ]:
filename = 'X_train'
outfile = open(filename,'wb')

pickle.dump(X_train,outfile)
outfile.close()

In [ ]:
filename = 'Y_train'
outfile = open(filename,'wb')

pickle.dump(y,outfile)
outfile.close()

In [ ]:
filename = 'X_valid'
outfile = open(filename,'wb')

pickle.dump(X_valid,outfile)
outfile.close()

In [ ]:
filename = 'X_test'
outfile = open(filename,'wb')

pickle.dump(X_test,outfile)
outfile.close()

In [ ]:
filename = 'Y_valid'
outfile = open(filename,'wb')

pickle.dump(Y_valid,outfile)
outfile.close()

In [ ]:
filename = 'all_data'
outfile = open(filename,'wb')

pickle.dump(all_data,outfile)
outfile.close()

### Read the pickled files in again

Since we have saved off the files to our local directory, we can restart the kernel if needed and just bring in the dataframes we need.



In [3]:
filename = 'X_train'
infile = open(filename,'rb')
X_train = pickle.load(infile)
infile.close()

In [4]:
filename = 'X_valid'
infile = open(filename,'rb')
X_valid = pickle.load(infile)
infile.close()

In [5]:
filename = 'Y_train'
infile = open(filename,'rb')
y = pickle.load(infile)
infile.close()

In [6]:
filename = 'Y_valid'
infile = open(filename,'rb')
Y_valid = pickle.load(infile)
infile.close()

In [7]:
filename = 'X_test'
infile = open(filename,'rb')
X_test = pickle.load(infile)
infile.close()

In [8]:
## For whatever reason the test data got shuffled. So we will read the test file in again, merge it with the X_test data
## to get the proper order of the ID's
test = pd.read_csv('test.csv.gz')
TEST_ID = test['ID']
a = pd.merge(X_test, test, left_on=['shop_id','item_id'],right_on=['shop_id','item_id'])
test_id_to_be_used= a['ID']

## Modeling

### LightGBM

LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

- Faster training speed and higher efficiency.
- Lower memory usage.
- Better accuracy.
- Support of parallel and GPU learning.
- Capable of handling large-scale data.
- For more details, please refer to Features.


LightGBM really good at handling datasets larger than 100K records, and does so realatively fast compared to XGBoost. 

https://lightgbm.readthedocs.io/en/latest/

In [ ]:
## Transform train and validation into lgb dataset structures required for modeling.
lgb_train = lgb.Dataset(X_train, y)
lgb_eval = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)

Like most boosted models, we will need to tune our hyper-paremeters. These are the ones that I had the most success with, but it does not mean that they are the "ultimate" ones.

Generally speaking, I follow the guidance of the documentations:

LightGBM uses the leaf-wise tree growth algorithm, while many other popular tools use depth-wise tree growth. Compared with depth-wise growth, the leaf-wise algorithm can converge much faster. However, the leaf-wise growth may be over-fitting if not used with the appropriate parameters.

To get good results using a leaf-wise tree, these are some important parameters:

__num_leaves.__ This is the main parameter to control the complexity of the tree model. Theoretically, we can set num_leaves = 2^(max_depth) to obtain the same number of leaves as depth-wise tree. However, this simple conversion is not good in practice. The reason is that a leaf-wise tree is typically much deeper than a depth-wise tree for a fixed number of leaves. Unconstrained depth can induce over-fitting. Thus, when trying to tune the num_leaves, we should let it be smaller than 2^(max_depth). For example, when the max_depth=7 the depth-wise tree can get good accuracy, but setting num_leaves to 127 may cause over-fitting, and setting it to 70 or 80 may get better accuracy than depth-wise.

__min_data_in_leaf.__ This is a very important parameter to prevent over-fitting in a leaf-wise tree. Its optimal value depends on the number of training samples and num_leaves. Setting it to a large value can avoid growing too deep a tree, but may cause under-fitting. In practice, setting it to hundreds or thousands is enough for a large dataset.
max_depth. You also can use max_depth to limit the tree depth explicitly.

In [ ]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads' : 4
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(Y_valid, y_pred) ** 0.5)

In [ ]:
num_features = 50
indxs = np.argsort(gbm.feature_importance())[:num_features]
    
feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance()[indxs],X_train.columns[indxs])), columns=['Value','Feature'])

plt.figure(figsize=(20, 20))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('Top {} LightGBM Features accorss folds'.format(num_features))
plt.tight_layout()
plt.show()

Now we are ready to submit!

In [ ]:
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
Y_pred = Y_pred.clip(0,20)
submission = pd.DataFrame({
    "ID": test_id_to_be_used, 
    "item_cnt_month": Y_pred
})
submission.to_csv('lgb_submission 7.csv', index=False)


Thank you for reading this far! If you like what you saw, please upvote or comment!

### XGBOOST

In [ ]:
ts = time.time()

model = XGBRegressor(
 #   max_depth=3,
 #   n_estimators=1000,
 #   min_child_weight=300, 
  #  colsample_bytree=0.8, 
  #  subsample=0.8, 
  eta=0.3,    
  #  seed=42,
    nthreads = 4)

model.fit(
    X_train, 
    y, 
    eval_metric="rmse", 
    eval_set=[(X_train, y), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 20)

time.time() - ts


In [ ]:
Y_pred = model.predict(X_test)
Y_pred = Y_pred.clip(0,20)
submission = pd.DataFrame({
    "ID": test_id_to_be_used, 
    "item_cnt_month": Y_pred
})
submission.to_csv('xgb_submission v7.csv', index=False)


## Stacked Models

In [9]:
# defining error functions for handy use. 

kfolds = KFold(n_splits=10, shuffle=False, random_state=4202)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X_train):
    rmse = np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [10]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [11]:
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds, tol=0.001))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))

In [12]:
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                      


In [13]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

In [14]:
xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

In [15]:
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [16]:
from datetime import datetime

In [17]:
## Iterate over each of the model, calculate Average RMSE, STD of RMSE and time to run

models = (ridge, lasso, elasticnet, gbr, xgboost, lightgbm)
models_str =('ridge', 'lasso', 'elasticnet', 'xgboost', 'lightgbm')
for model_name,model in zip(models_str, models):    
    start_time= datetime.now()
    score = cv_rmse(model , X_train)
    print("{} RMSE: {:.4f} STD: ({:.4f}) Time:".format(str(model_name),score.mean(), score.std()), datetime.now()-start_time, )

ridge RMSE: 0.8254 STD: (0.3101) Time: 3:22:23.111299
lasso RMSE: 0.8255 STD: (0.3103) Time: 7:12:27.014321
elasticnet RMSE: 0.8254 STD: (0.3103) Time: 8:06:15.958091
xgboost RMSE: 0.8197 STD: (0.3244) Time: 2 days, 21:15:18.372290
[07:19:42] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[23:05:37] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[14:46:16] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[06:30:19] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[22:16:47] Tree method is automatically selected to be 'approx' for fast

In [ ]:

print('START Fit')

print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X_train), np.array(y))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(X_train, y)

print('Lasso')
lasso_model_full_data = lasso.fit(X_train, y)

print('Ridge')
ridge_model_full_data = ridge.fit(X_train, y)


print('xgboost')
xgb_model_full_data = xgboost.fit(X_train,y)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(X_train, y)

START Fit
stack_gen


In [ ]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.1 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
#            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.2 * xgb_model_full_data.predict(X)) + \
            (0.2 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))

In [ ]:
print('RMSLE score on validation data:')
print(rmsle(Y_valid, blend_models_predict(X_valid)))

In [ ]:
Y_pred = blend_models_predict(X_test)
Y_pred = Y_pred.clip(0,20)
submission = pd.DataFrame({
    "ID": test_id_to_be_used, 
    "item_cnt_month": Y_pred
})
submission.to_csv('blended_submission.csv', index=False)

## To Do
 - Test SET
     - There are items that are new in the test set. How to handle?
     
 - What about stale items? Should I just set them to zero if they haven't had a sale in say 6 months?
 
 
 - Run the Ensemble models
 
 - Add Features:
 - Discounts
 - Shop Revenue
 
 - 
 
 
 
 

# SCRATCH CODE BELOW

In [ ]:
filename = 'all_data'
infile = open(filename,'rb')
all_data = pickle.load(infile)
infile.close()

In [ ]:
## allows us to pick up the european formatting of the dates in the trainset
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y') 
# importing the trainset with dates correctly formatted
sales = pd.read_csv('sales_train.csv.gz', parse_dates = ['date'], date_parser = dateparse)

In [ ]:
ts = time.time()
group = sales.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group, on=['item_id'], how='left')
all_data['item_avg_item_price'] = all_data['item_avg_item_price'].astype(np.float16)

group = sales.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

all_data = pd.merge(all_data, group, on=['date_block_num','item_id'], how='left')
all_data['date_item_avg_item_price'] = all_data['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
all_data = lag_feature(all_data, lags, 'date_item_avg_item_price')

for i in lags:
    all_data['delta_price_lag_'+str(i)] = (all_data['date_item_avg_item_price_lag_'+str(i)] - all_data['item_avg_item_price']) / all_data['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
all_data['delta_price_lag'] = all_data.apply(select_trend, axis=1)
all_data['delta_price_lag'] = all_data['delta_price_lag'].astype(np.float16)
all_data['delta_price_lag'].fillna(0, inplace=True)

In [ ]:
all_data.head()

In [ ]:
## From:

## https://www.kaggle.com/kyakovlev/1st-place-solution-part-1-hands-on-data
no_data_items = X_test[~(X_test['item_id'].isin(X_train['item_id']))]
len(no_data_items['item_id'].unique())